In [279]:
using JuMP
using SDPA
using SCIP
using LinearAlgebra
using SCS

# $\zeta | pentagon | = \sqrt{5}$

In [280]:
model = Model(SCS.Optimizer)

A JuMP Model
├ solver: SCS
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

In [281]:
n = 5
edges = [(1,2), (2,3), (3,4), (4,5), (5,1)]

# n = 3
# edges = [(1,2), (2,3)]

# adjec = zeros(5, 5)

# for i in 1:n
#     for j in 1:n
#         adjec[i,j] = n * (i-1) + (j-1)
#     end
# end

# adjec

5-element Vector{Tuple{Int64, Int64}}:
 (1, 2)
 (2, 3)
 (3, 4)
 (4, 5)
 (5, 1)

In [282]:
adj = falses(n,n)

for (i,j) in edges
    adj[i,j] = true
    adj[j,i] = true
end

adj

5×5 BitMatrix:
 0  1  0  0  1
 1  0  1  0  0
 0  1  0  1  0
 0  0  1  0  1
 1  0  0  1  0

In [283]:
@variable(model, Γ[1:(n+1), 1:(n+1)], PSD)

6×6 Symmetric{VariableRef, Matrix{VariableRef}}:
 Γ[1,1]  Γ[1,2]  Γ[1,3]  Γ[1,4]  Γ[1,5]  Γ[1,6]
 Γ[1,2]  Γ[2,2]  Γ[2,3]  Γ[2,4]  Γ[2,5]  Γ[2,6]
 Γ[1,3]  Γ[2,3]  Γ[3,3]  Γ[3,4]  Γ[3,5]  Γ[3,6]
 Γ[1,4]  Γ[2,4]  Γ[3,4]  Γ[4,4]  Γ[4,5]  Γ[4,6]
 Γ[1,5]  Γ[2,5]  Γ[3,5]  Γ[4,5]  Γ[5,5]  Γ[5,6]
 Γ[1,6]  Γ[2,6]  Γ[3,6]  Γ[4,6]  Γ[5,6]  Γ[6,6]

In [284]:
# @constraint(model, Γ .>= 0)

In [285]:
@constraint(model, Γ[1,1] == 1.0)

Γ[1,1] == 1

In [286]:
for i in 1:n
    @constraint(model, Γ[1, i+1] == Γ[i+1, i+1])
end

In [287]:
for i in 1:n
    for j in i:n
        if adj[i,j]
            @constraint(model, Γ[i+1, j+1] == 0.0)
            @constraint(model, Γ[j+1, i+1] == 0.0)
        end
    end
end

In [288]:
@objective(model, Max, sum( Γ[i+1, i+1] for i in 1:n ))

Γ[2,2] + Γ[3,3] + Γ[4,4] + Γ[5,5] + Γ[6,6]

In [289]:
optimize!(model)

------------------------------------------------------------------
	       SCS v3.2.8 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 21, constraints m: 37
cones: 	  z: primal zero / dual free vars: 16
	  s: psd vars: 21, ssize: 1
settings: eps_abs: 1.0e-004, eps_rel: 1.0e-004, eps_infeas: 1.0e-007
	  alpha: 1.50, scale: 1.00e-001, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-006
	  acceleration_lookback: 10, acceleration_interval: 10
	  compiled with openmp parallelization enabled
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 42, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0|8.73e+000 8.46e-001 4.52e+001 -2.25e+001 1.00e-001 2.27e-004 
   125|1.23e-005 6.34e-006 1.20e-005 -2.24e+

In [290]:
value.(Γ)

6×6 Matrix{Float64}:
 1.0       0.447223    0.447223    0.447223    0.447223    0.447223
 0.447223  0.447223    2.48544e-9  0.276405    0.276405    2.48544e-9
 0.447223  2.48544e-9  0.447223    2.48544e-9  0.276405    0.276405
 0.447223  0.276405    2.48544e-9  0.447223    2.48544e-9  0.276405
 0.447223  0.276405    0.276405    2.48544e-9  0.447223    2.48544e-9
 0.447223  2.48544e-9  0.276405    0.276405    2.48544e-9  0.447223

# $\zeta | G | = \sqrt{5}$

In [291]:
objective_value(model)

2.2361155160552033

# $\zeta | G x H | \ge 4$

In [292]:
objective_value(model) * objective_value(model)

5.000212601142828